# ПОДЗАПРОСЫ, ПРЕДСТАВЛЕНИЯ, СОЗДАНИЕ ТАБЛИЦ

Далее в заданиях используются данные с платформы Airbnb в Берлине ✈️.

Имеются следующие таблицы:
    
1. `listings` – информация о жилье, включая полные описания, характеристики и средние оценки в отзывах; поскольку столбцов очень много, нужные перечислены в текстах самих задач;  
2. `calendar_summary` – информация о доступности и цене того или иного жилья по дням  
    listing_id – идентификатор жилья (объявления)  
    date       – дата  
    available  – доступность жилья в данный день (t/f)  
    price      – цена за ночь  

3. `reviews` – отзывы  
    listing_id   –  идентификатор жилья      
    id  – id отзыва   
    date  – дата отзыва      
    reviewer_id  – id ревьюера (автора отзыва)  
    reviewer_name  – имя автора  
    comments – сам отзыв  

### 1

Подзапросы можно использовать в разных частях запроса, в том числе и в блоке WHERE. Потренируемся!

Сначала оставьте только те объявления, в которых оценка на основе отзывов выше среднего, а число отзывов в месяц составляет строго меньше трёх. Затем отсортируйте по убыванию две колонки: сначала по числу отзывов в месяц, потом по оценке. В качестве ответа укажите id объявления из первой строки.

· review_scores_rating – оценка на основе отзывов  
· reviews_per_month – число отзывов в месяц  
· id – идентификатор объявления

In [ ]:
SELECT id, 
       toFloat64OrNull(review_scores_rating) AS review_scores_rating,
       reviews_per_month 
  FROM listings 
 WHERE review_scores_rating > (SELECT AVG(toFloat64OrNull(review_scores_rating))
                                 FROM listings) 
   AND reviews_per_month < 3
 ORDER BY reviews_per_month DESC,
         review_scores_rating DESC
 LIMIT 100

### 2

Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, для которых расстояние оказалось меньше среднего. Результат отсортируйте по убыванию, тем самым выбрав комнату, которая является наиболее удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем. 

· id – идентификатор объявления  
· host_id – идентификатор хозяина  
· room_type – тип жилья ('Private room')  
· latitude – широта  
· longitude – долгота  
· 52.5200 с.ш., 13.4050 в.д – координаты центра Берлина  

In [ ]:
SELECT id,
       host_id,
       latitude,
       longitude,
       geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance
  FROM listings
 WHERE distance < (SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude))) 
                     FROM listings 
                    WHERE room_type = 'Private room')
 ORDER BY distance DESC
 LIMIT 10

### 3

Представим, что вы планируете снять жилье в Берлине на 7 дней, используя более хитрые фильтры, чем предлагаются на сайте.

В этой задаче можно потренироваться в написании подзапросов, но задание можно решить и без них.

Отберите объявления из таблицы listings, которые:

· находятся на расстоянии от центра меньше среднего (здесь нам пригодится запрос из предыдущего задания)  
· обойдутся дешевле 100$ в день (price с учетом cleaning_fee, который добавляется к общей сумме за неделю, т.е его нужно делить на кол-во дней)  
· имеют последние отзывы (last_review), начиная с 1 сентября 2018 года  
· имеют WiFi в списке удобств (amenities)  
Отсортируйте полученные значения по убыванию review_scores_rating (не забудьте перевести строку к численному виду) и в качестве ответа укажите host_id из первой строки. 

In [ ]:
SELECT id,
       host_id,
       geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance,
       toFloat32OrNull(review_scores_rating) AS review_scores_rating,
       price,
       cleaning_fee,
       toFloat64OrNull(replaceRegexpAll(price,'[$,]','')) + toFloat64OrNull(replaceRegexpAll(cleaning_fee,'[$,]','')) / 7 AS price_per_week,
       last_review,
       amenities
  FROM listings
 WHERE distance < (SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)))
                     FROM listings)
   AND price_per_week < 100
   AND last_review >= '2018-09-01'
   AND multiSearchAnyCaseInsensitive(amenities, ['wifi']) != 0
 ORDER BY review_scores_rating DESC
 LIMIT 10

### 4

Давайте найдем в таблице calendar_summary те доступные (available='t') объявления, у которых число отзывов от уникальных пользователей в таблице reviews выше среднего.

Для этого с помощью конструкции WITH посчитайте среднее число уникальных reviewer_id из таблицы reviews на каждое жильё, потом проведите джойн таблиц calendar_summary и reviews по полю listing_id (при этом из таблицы calendar_summary должны быть отобраны уникальные listing_id, отфильтрованные по правилу available='t'). Результат отфильтруйте так, чтобы остались только записи, у которых число отзывов от уникальных людей выше среднего.

Отсортируйте результат по возрастанию listing_id и в качестве ответа впишите количество отзывов от уникальных пользователей из первой строки.

In [ ]:
WITH (SELECT AVG(review_num)
        FROM (SELECT listing_id,
                     COUNT (DISTINCT reviewer_id) AS review_num
                FROM reviews
               GROUP BY listing_id
     )) AS AVG_review_num
SELECT listing_id,
       review_num
  FROM (SELECT listing_id
          FROM calendar_summary
         WHERE available = 't'
         GROUP BY listing_id
       ) AS l
  JOIN 
       (SELECT listing_id,
               COUNT(DISTINCT reviewer_id) AS review_num
          FROM reviews
         GROUP BY listing_id
       ) AS r
    ON l.listing_id = r.listing_id
 WHERE review_num > AVG_review_num
 ORDER BY listing_id
 LIMIT 10

Возвращаемся к данными мобильного приложения!

`installs` — содержит данные об установках приложения по дням.

· DeviceID — идентификатор устройства, на которое было установлено приложение;  
· InstallationDate — дата установки приложения;  
· InstallCost — цена установки приложения в рублях;  
· Platform — платформа, на которой было установлено приложение (iOS/ Android);  
· Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).  

`events` — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.  

· DeviceID — идентификатор устройства, на котором используется приложение;  
· AppPlatform — платформа, на которой используется приложение (iOS/ Android);  
· EventDate — дата, за которую собрана статистика;  
· events — количество просмотров всех товаров за этот день у этого DeviceID.  

`checks` — содержит данные о покупках пользователей в приложении по дням  
  
· UserID — идентификатор пользователя;  
· Rub — суммарный чек пользователя на дату;  
· BuyDate — дата, за которую собрана статистика.  
   
`devices` – чтобы просмотры и установки можно было объединить с покупками  

· DeviceID — идентификатор устройства;  
· UserID — идентификатор пользователя.  

### 5

С помощью оператора CASE в SQL можно составлять условные конструкции, выполнять проверку условий и возвращать результат в зависимости от их выполнения. Самих условий под оператором WHEN может быть сколько угодно, конструкцию необходимо завершить оператором END.

Какие из перечисленных условных конструкций составлены некорректно?

__· WHERE price > 1000 THEN 'ok' ELSE 'not ok' END__   
__· CASE WHEN price > 1000 THEN 'ok' ELSE 'not ok'__    
· CASE WHEN price>1000 THEN 'ok' WHEN price<500 THEN 'not ok' ELSE 'normal' END

### 6

Теперь задача! Используйте таблицу checks и разделите всех покупателей на сегменты:

NB! Правые границы берутся не включительно, например, чек в 10 рублей будет относиться к сегменту С

А — средний чек покупателя менее 5 ₽  
B — средний чек покупателя от 5-10 ₽  
C — средний чек покупателя от 10-20 ₽    
D — средний чек покупателя от 20 ₽  
Отсортируйте результирующую таблицу по возрастанию UserID и укажите сегмент четвертого пользователя.

In [ ]:
SELECT UserID, 
       AVG(Rub) AS AVG_check,
       CASE WHEN AVG_check < 5 THEN 'A'
            WHEN 5 <= AVG_check AND AVG_check < 10 THEN 'B'
            WHEN 10 <= AVG_check AND AVG_check < 20 THEN 'C'
            ELSE 'D'
       END AS abcd
  FROM checks
 GROUP BY UserID
 ORDER BY UserID 
 LIMIT 10

### 7

Используйте предыдущий запрос как подзапрос и посчитайте, сколько клиентов приходится на каждый сегмент и сколько доходов он приносит. Отсортируйте результат по убыванию суммы доходов на сегмент и в качестве ответа укажите наибольшую сумму.

In [ ]:
SELECT segment, 
       COUNT(UserID) AS users,
       SUM(SUM_rub) AS SUM_per_segment
  FROM (SELECT UserID, 
               AVG(Rub) AS AVG_check,
               SUM(Rub) AS SUM_rub,
               CASE WHEN AVG_check < 5 THEN 'A'
                    WHEN 5 <= AVG_check AND AVG_check < 10 THEN 'B'
                    WHEN 10 <= AVG_check AND AVG_check < 20 THEN 'C'
                    ELSE 'D'
                END AS segment
          FROM checks
         GROUP BY UserID
         ORDER BY UserID 
       )
 GROUP BY segment
 ORDER BY SUM_per_segment DESC

### 8

Вернемся к таблице AirBnb. Предположим, что в выборе жилья нас интересует только два параметра: наличие кухни (kitchen) и гибкой системы отмены (flexible), причем первый в приоритете.

Создайте с помощью оператора CASE колонку с обозначением группы, в которую попадает жилье из таблицы listings:

1. 'good', если в удобствах (amenities) присутствует кухня и система отмены (cancellation_policy) гибкая
2. 'ok', если в удобствах есть кухня, но система отмены не гибкая
3. 'not ok' во всех остальных случаях  

Результат отсортируйте по новой колонке по возрастанию, установите ограничение в 5 строк, в качестве ответа укажите host_id первой строки.

Обратите внимание, что cancellation_policy - это отдельная колонка, по ней необходимо смотреть систему отмены

In [ ]:
SELECT host_id,
       CASE WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen']) 
             AND cancellation_policy = 'flexible' THEN 'good'
            WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen']) 
             AND cancellation_policy != 'flexible' THEN 'ok'
            ELSE 'not ok'
        END AS needs
  FROM listings
 ORDER BY needs 
 LIMIT 5

### 9

Напишите запрос для создания таблицы со следующими параметрами, также подобрав подходящий тип данных.

Название таблицы: `reviews`  
База данных: test  
Столбцы:
- listing_id – идентификатор объявления, может быть только положительным и целым числом, 32-битный тип данных

- id – идентификатор хозяина, может быть только положительным и целым числом, 32-битный тип данных

- created_at – дата со временем (2020-01-01 00:00:00), часовой пояс – 'Europe/Moscow'

- reviewer_id – идентификатор ревьюера, может быть только положительным и целым числом, 32-битный тип данных

- reviewer_name – имя того, кто оставил отзыв

- comments - текст отзыва

Движок: MergeTree  
Сортировка: listing_id, id  

In [ ]:
CREATE TABLE test.reviews (listing_id UInt32,
                                   id UInt32,
                                 date DateTime('Europe/Moscow'),
                          reviewer_id UInt32,
                        reviewer_name String,
                             comments String
                          )
Engine = MergeTree 
ORDER BY (listing_id, id) 

### 10

К вам пришел коллега с новостями: оказывается, в поле date будет записываться только дата, без времени, поэтому нужно изменить тип данных!

Напишите запрос, который модифицирует тип данных, и введите его в поле ниже без кавычек и лишних пробелов. Не забудьте указать базу данных test перед названием таблицы! 

In [ ]:
ALTER TABLE test.reviews MODIFY COLUMN date Date

### 11

Предположим, ваш коллега вставил данные, но что-то перепутал. Часть строк с комментариями осталась совершенно пустой! 

Напишите запрос, который удалит из таблицы test.reviews те строки, где в столбце comments встречаются пустые значения (''). Введите его в поле ниже без кавычек и лишних пробелов.

In [ ]:
ALTER TABLE test.reviews DELETE WHERE comments=''

### 12

С помощью какого запроса можно создать обычное представление над таблицей test.reviews, которое будет содержать все записи из test.reviews, сгруппированные по reviewer_id с подсчитанным количеством отзывов (id) на каждого пользователя?

In [ ]:
CREATE VIEW test.reviews_number AS (SELECT reviewer_id, COUNT(id) reviews_count FROM test.reviews GROUP BY reviewer_id)

### 13

С помощью какого запроса можно создать новый столбец reviewer_score в таблице reviews после столбца reviewer_name?

In [ ]:
ALTER TABLE test.reviews ADD COLUMN reviewer_score UInt8 AFTER reviewer_name

### 14

Напишите запрос для добавления в таблицу test.reviews колонки price после колонки comments, которая может быть числом с плавающей точкой, 32-битный тип данных.

In [ ]:
ALTER TABLE test.reviews ADD COLUMN price Float32 AFTER comments

#### 15

Напишите запрос, который удвоит price для всех строк с датой (date) после 2019-01-01.

Обратите внимание, что для сравнения с датой в формате '2019-01-01' можно не преобразовывать данные в колонке date.

In [ ]:
ALTER TABLE test.reviews UPDATE price=price*2 WHERE date>'2019-01-01'